В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

# Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

# Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [27]:
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [12]:
digits = load_digits()
index = int(len(digits.data) * 0.75)
x_train = digits.data[0:index]
y_train = digits.target[0:index]
x_test = digits.data[index:]
y_test = digits.target[index:]

print(digits.data.shape)
print(x_train.shape)
print(x_test.shape)

(1797, 64)
(1347, 64)
(450, 64)


In [40]:
class KNN:
    def fit(self, X, y):
        self.X = X
        self.y = y
        
    
    def find_closest(self, x):
        distances = ((self.X - x) ** 2).sum(axis = 1)
        min_d = distances[0]
        min_t = self.y[0]
        for d, t in zip(distances, self.y):
            if d < min_d:
                min_d = d
                min_t = t
        return min_t
    
    
    def predict(self, test):    
        predictions = []
        for t in test:
            predictions.append(self.find_closest(t))
        return predictions
    
        
knn = KNN()
knn.fit(x_train, y_train)
prediction_knn = knn.predict(x_test)
error = 1 - accuracy_score(y_test, prediction_knn)
with open('submissions/3_knn_vs_random_forest_1.txt', 'w') as fd:
    fd.write(str(error))
error

0.037777777777777799

In [41]:
rf_clf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf_clf.fit(x_train, y_train)
prediction_rf = rf_clf.predict(x_test)
error = 1 - accuracy_score(y_test, prediction_rf)
with open('submissions/3_knn_vs_random_forest_2.txt', 'w') as fd:
    fd.write(str(error))
error

0.068888888888888888